FRI-based PCS

## How FRI protocol works
A FRI protocol is a protocol for proving that a function $f:H \to \mathbf{F}$ is closed to a polynomial of low degree $d$, where $d \ll |H|$. 
It can be divided into two phases: commit and query.
In the commit phash, the prover commits to (via Merkle trees) a series of functions generated from $f$ and random elements $v_0, v_1, ... $ from $\mathbf{K}$ provided by the verifier at each round. 
Then in the query phase, the prover provides a set of evaluations of the previously committed functions at a point randomly chosen by the verifier.

### Commit Phase
Denote by $p_0$ the function of low degree.

### Query Phase



## Codes
There are 8 files in Plonky3/fri/src: config.rs, fold_even_odd.rs, hiding_pcs.rs, proof.rs, prover.rs, two_adic_pcs, and verifier.rs.

### config.rs
There are two main functions in this code: FriConfig and FriGenericConfig.
FriConfig defines the configuration parameters for the FRI protocol, including the blow-up factor (log_blowup), the number of queries(num_queries), and anti-attack parameters(proof_of_work_bits).
FriGenericConfig provides an abstract interface for specific implementations of FRI, including fold_row and fold_matrix for folding opreation.

### fold_even_odd.rs
This code implements a folding operation the fold_even_odd function:
$p(x) = p_{even}(x^2) + x p_{odd}(x^2)$

### hiding_pcs.rs
This code implements a hiding pcs by adding random codewords.

### proof.rs
combine communications into proofs.
1. FriProof
2. QueryProof
3. CommitPhaseProofStep

### Prover.rs
function Prove.
main stage: commit -> challenge -> query ->pack proofs

commit
answer_query

### two_adic_pcs.rs
FRI over two-adic field.

1. define sturct TwoAdicFriPcs
2. a new TwoAdicFriPcs instance
---------def-----------
3. Generic config
4. fold_row
5. fold_matrix  
--------config----------
6. interface for TwoAdicFriPcs
    - natural_domain_for_degree
    - commit

### verifier.rs
function verify verify_query